In [12]:
import os
import pandas as pd
import optuna
import numpy as np
import openml
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import plotly
import scipy.stats as stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from optuna.samplers import TPESampler, CmaEsSampler
from xgboost import XGBClassifier
from plotly.io import show
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import roc_auc_score

In [13]:
dataset_ids = [
    1590,   # Adult
    1510,   # Breast Cancer Wisconsin (Diagnostic)
    1461,   # Bank Marketing
    24,     # Mushroom
    40945   # Titanic
]

In [14]:
def fetch_and_prepare(openml_id):
    global y
    global X
    dataset = openml.datasets.get_dataset(openml_id)
    print(f">>> {dataset.name} (ID: {openml_id})")
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')
    for col in X.select_dtypes(include=['category', 'object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    #scaler = MinMaxScaler()
    #X_imputed = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)
    if y.dtype == 'category':
        y = LabelEncoder().fit_transform(y)
    zbior = X_imputed['Target'] = y
    return X_imputed

In [15]:
zbior = fetch_and_prepare(1590)

>>> adult (ID: 1590)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(zbior.iloc[:, :-1], zbior['Target'], test_size=0.30, random_state=42)

def objective(trial):
             
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])
    if booster == "gblinear":
        n_estimators = trial.suggest_int("n_estimators", 921, 4551)
        eta = trial.suggest_float("eta", 0.002, 0.355, log=True)
        lambda_ = trial.suggest_float("lambda", 0.008, 29.755, log=True)     
        alpha = trial.suggest_float("alpha", 0.002, 6.105, log=True)
    else:
        n_estimators = trial.suggest_int("n_estimators", 921, 4551)          
        eta = trial.suggest_float("eta", 0.002, 0.355, log=True)             
        subsample = trial.suggest_float("subsample", 0.545, 0.958)  
        max_depth = trial.suggest_int("max_depth", 6, 14)
        min_child_weight = trial.suggest_float("min_child_weight", 1.295, 6.984)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.419, 0.864)  
        colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.335, 0.886)
        lambda_ = trial.suggest_float("lambda", 0.008, 29.755, log=True)     
        alpha = trial.suggest_float("alpha", 0.002, 6.105, log=True)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    wyniki = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        X_train_fold = X_train_fold.reset_index(drop=True)
        X_test_fold = X_test_fold.reset_index(drop=True)
        y_train_fold = y_train_fold.reset_index(drop=True)
        y_test_fold = y_test_fold.reset_index(drop=True)

        X_train_fold = list(zip(*[X_train_fold[col] for col in X_train_fold]))
        X_test_fold = list(zip(*[X_test_fold[col] for col in X_test_fold]))

        XGBoost = XGBClassifier(n_estimators=n_estimators,
        eta=eta, subsample=subsample, booster=booster, max_depth=max_depth, min_child_weight=min_child_weight, colsample_bytree=colsample_bytree,
        colsample_bylevel=colsample_bylevel,
        reg_lambda=lambda_, reg_alpha=alpha, random_state=42, n_jobs=-1, eval_metric="auc")


        model = XGBoost.fit(X_train_fold, y_train_fold)


        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        auc = roc_auc_score(y_test_fold, y_pred_proba)
                                            
        wyniki.append(auc)

    rezultat = np.mean(wyniki)
    return rezultat

In [17]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
wynik = study.optimize(objective, n_trials=32)

[I 2025-11-12 20:52:05,660] A new study created in memory with name: no-name-0fb22780-5ff6-415c-a8a0-3b37773a4b8d
[W 2025-11-12 22:25:42,477] Trial 0 failed with parameters: {'booster': 'dart', 'n_estimators': 1710, 'eta': 0.3450834885243372, 'subsample': 0.6574620703642793, 'max_depth': 13, 'min_child_weight': 4.049507099046526, 'colsample_bytree': 0.5840683867598503, 'colsample_bylevel': 0.5422733421113989, 'lambda': 7.53408469940207, 'alpha': 0.007367552757549518} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Mikolaj Ozimek\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Mikolaj Ozimek\AppData\Local\Temp\ipykernel_12892\1143840741.py", line 43, in objective
    model = XGBoost.fit(X_train_fold, y_train_fold)
  File "C:\Users\Mikolaj Ozimek\anaconda3\Lib\site-packages\xgboost\core.py", line 774, in inner_f
    return func(**kwargs)
  File "C:\U

KeyboardInterrupt: 

In [10]:
study.best_value

0.8706308982450709

In [8]:
study.best_params

{'max_depth': 9,
 'learning_rate': 0.005774611041567384,
 'n_estimators': 1710,
 'subsample': 0.9762184215070683,
 'colsample_bytree': 0.7290221048349554,
 'min_child_weight': 6}